In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import json
import ast


# Step 1: Load and prepare the datasets

In [29]:

def load_data():
    customers = pd.read_csv('customers.csv')
    orders = pd.read_csv('orders.csv')
    dishes = pd.read_csv('dishes.csv')
    chefs = pd.read_csv('chefs.csv')
    
    # Convert string representations of lists to actual lists
    customers['specialties preference'] = customers['specialties preference'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    chefs['specialties'] = chefs['specialties'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    # Parse dish information in orders
    orders['dishes'] = orders['dishes'].apply(
        lambda x: json.loads(x.replace("'", "\"")) if isinstance(x, str) else x
    )
    
    return customers, orders, dishes, chefs


# Step 2: Content-based filtering for new customers

In [30]:
def content_based_recommendations(customer_id, customers, chefs, top_n=3):
    """
    Recommend chefs based on matching customer preferences with chef specialties.
    Used for new customers without order history.
    """
    customer = customers[customers['customer_id'] == customer_id].iloc[0]
    
    # Get customer's diet preference and cuisine specialties
    diet_pref = customer['preference']
    cuisine_prefs = customer['specialties preference']
    
    # Create a feature vector for each chef
    chef_features = []
    for _, chef in chefs.iterrows():
        # Calculate cuisine match score (how many customer preferences match chef specialties)
        cuisine_match = sum(cuisine in chef['specialties'] for cuisine in cuisine_prefs) if len(cuisine_prefs) > 0 else 0
        cuisine_match_ratio = cuisine_match / len(cuisine_prefs) if len(cuisine_prefs) > 0 else 0
        
        # Consider chef rating and experience
        chef_score = (chef['averageRating'] / 5) * 0.5 + (chef['experience'] / 10) * 0.2 + cuisine_match_ratio * 0.3
        
        chef_features.append({
            'chef_id': chef['chef_id'],
            'name': chef['name'],
            'score': chef_score,
            'cuisine_match': cuisine_match_ratio,
            'rating': chef['averageRating']
        })
    
    # Sort chefs by score
    chef_features.sort(key=lambda x: x['score'], reverse=True)
    
    return chef_features[:top_n]


# Step 3: Create a user-item interaction matrix for collaborative filtering

In [31]:
def create_user_item_matrix(orders, customers, chefs):
    """
    Create a matrix where rows are customers and columns are chefs.
    Each cell contains the number of times a customer ordered from a chef.
    """
    # Count orders for each customer-chef pair
    customer_chef_counts = orders.groupby(['customer_id', 'chef_id']).size().reset_index(name='order_count')
    
    # Create pivot table
    user_item_matrix = customer_chef_counts.pivot(
        index='customer_id', 
        columns='chef_id', 
        values='order_count'
    ).fillna(0)
    
    return user_item_matrix


# Step 4: Collaborative filtering for existing customers

In [32]:
def collaborative_filtering(customer_id, user_item_matrix, customers, chefs, top_n=3):
    """
    Recommend chefs based on customer's previous orders and similarity to other customers.
    Used for existing customers with order history.
    """
    # Check if customer exists in matrix
    if customer_id not in user_item_matrix.index:
        return []
    
    # Calculate cosine similarity between users
    user_similarity = cosine_similarity(user_item_matrix)
    user_similarity_df = pd.DataFrame(
        user_similarity,
        index=user_item_matrix.index,
        columns=user_item_matrix.index
    )
    
    # Get similar users to our target customer
    similar_users = user_similarity_df[customer_id].sort_values(ascending=False)[1:6]  # Top 5 similar users
    
    # Which chefs has the customer already ordered from?
    customer_chefs = user_item_matrix.loc[customer_id]
    customer_chefs = set(customer_chefs[customer_chefs > 0].index)
    
    # Collect recommendations from similar users
    chef_scores = {}
    for similar_user, similarity in similar_users.items():
        if similarity <= 0:  # Skip negatively correlated users
            continue
            
        # Get chefs this similar user has ordered from
        user_chefs = user_item_matrix.loc[similar_user]
        user_chefs = set(user_chefs[user_chefs > 0].index)
        
        # Recommend chefs this similar user has ordered from but our target customer hasn't
        for chef_id in user_chefs - customer_chefs:
            if chef_id not in chef_scores:
                chef_scores[chef_id] = 0
            chef_scores[chef_id] += similarity * user_item_matrix.loc[similar_user, chef_id]
    
    # Sort chefs by score and get top_n
    recommended_chefs = sorted(chef_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    # Get chef details
    chef_details = []
    for chef_id, score in recommended_chefs:
        chef = chefs[chefs['chef_id'] == chef_id].iloc[0]
        chef_details.append({
            'chef_id': chef_id,
            'name': chef['name'],
            'score': score,
            'rating': chef['averageRating'],
            'specialties': chef['specialties']
        })
    
    return chef_details


# Step 5: Hybrid recommendation system

In [33]:
def get_chef_recommendations(customer_id, customers, orders, dishes, chefs, top_n=3):
    """
    Main recommendation function that decides between content-based and collaborative filtering
    based on whether the customer has previous orders.
    """
    # Create user-item matrix for collaborative filtering
    user_item_matrix = create_user_item_matrix(orders, customers, chefs)
    
    # Check if customer has previous orders
    customer_orders = orders[orders['customer_id'] == customer_id]
    has_order_history = len(customer_orders) > 0
    
    if has_order_history:
        # Use collaborative filtering for existing customers
        print(f"Customer {customer_id} has order history. Using collaborative filtering.")
        recommendations = collaborative_filtering(customer_id, user_item_matrix, customers, chefs, top_n)
        
        # If collaborative filtering doesn't yield enough recommendations, supplement with content-based
        if len(recommendations) < top_n:
            content_recommendations = content_based_recommendations(customer_id, customers, chefs, top_n)
            # Add content recommendations not already in collaborative recommendations
            existing_ids = {rec['chef_id'] for rec in recommendations}
            for rec in content_recommendations:
                if rec['chef_id'] not in existing_ids and len(recommendations) < top_n:
                    recommendations.append(rec)
    else:
        # Use content-based filtering for new customers
        print(f"Customer {customer_id} is new. Using content-based filtering.")
        recommendations = content_based_recommendations(customer_id, customers, chefs, top_n)
    
    return recommendations



# Step 6: Advanced features - Incorporating dish preferences

In [34]:
def analyze_dish_preferences(customer_id, orders, dishes):
    """
    Analyze which categories and subcategories of dishes the customer prefers.
    """
    customer_orders = orders[orders['customer_id'] == customer_id]
    
    if len(customer_orders) == 0:
        return None
    
    # Extract all dish IDs from customer orders
    dish_ids = []
    for _, order in customer_orders.iterrows():
        order_dishes = order['dishes']
        for dish_item in order_dishes:
            dish_ids.append(dish_item['dish'])
    
    # Get dish details
    customer_dishes = dishes[dishes['dish_id'].isin(dish_ids)]
    
    # Count categories and subcategories
    category_counts = customer_dishes['category'].value_counts().to_dict()
    subcategory_counts = customer_dishes['subCategory'].value_counts().to_dict()
    
    return {
        'categories': category_counts,
        'subcategories': subcategory_counts
    }


# Step 7: Enhanced recommendation system with dish preferences

In [35]:
def enhanced_recommendations(customer_id, customers, orders, dishes, chefs, top_n=3):
    """
    Enhanced recommendation system that considers dish preferences.
    """
    base_recommendations = get_chef_recommendations(customer_id, customers, orders, dishes, chefs, top_n)
    dish_prefs = analyze_dish_preferences(customer_id, orders, dishes)
    
    if dish_prefs is None:
        return base_recommendations
    
    # Adjust chef scores based on their expertise in preferred dish categories
    for i, rec in enumerate(base_recommendations):
        chef_id = rec['chef_id']
        chef_dishes = dishes[dishes['chef_id'] == chef_id]
        
        # Calculate category match score
        category_match = 0
        total_categories = sum(dish_prefs['categories'].values())
        
        for category, count in dish_prefs['categories'].items():
            chef_category_count = len(chef_dishes[chef_dishes['category'] == category])
            if chef_category_count > 0:
                category_match += (count / total_categories) * (chef_category_count / len(chef_dishes))
        
        # Adjust recommendation score
        base_recommendations[i]['score'] = base_recommendations[i]['score'] * (1 + category_match)
    
    # Re-sort based on adjusted scores
    base_recommendations.sort(key=lambda x: x['score'], reverse=True)
    
    return base_recommendations[:top_n]


In [36]:
if __name__ == "__main__":
    customers, orders, dishes, chefs = load_data()
    
    # Example for an existing customer
    existing_customer_id = 'CUST001'
    recommendations = enhanced_recommendations(existing_customer_id, customers, orders, dishes, chefs)
    print(f"Recommendations for existing customer {existing_customer_id}:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['name']} (Score: {rec['score']:.2f}, Rating: {rec['rating']})")
    
    # Example for a new customer
    new_customer_id = 'CUST006'  # Assuming this customer has fewer or no orders
    recommendations = enhanced_recommendations(new_customer_id, customers, orders, dishes, chefs)
    print(f"\nRecommendations for new customer {new_customer_id}:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['name']} (Score: {rec['score']:.2f}, Rating: {rec['rating']})")
    
    # Get detailed insights for a customer
    print("\nDetailed dish preferences for existing customer:")
    dish_prefs = analyze_dish_preferences(existing_customer_id, orders, dishes)
    if dish_prefs:
        print("Preferred categories:", dish_prefs['categories'])
        print("Preferred subcategories:", dish_prefs['subcategories'])

Customer CUST001 has order history. Using collaborative filtering.
Recommendations for existing customer CUST001:
1. Fred Lara (Score: 1.28, Rating: 4.2)
2. Breanna Turner (Score: 0.67, Rating: 5.0)
3. Alexander Lopez (Score: 0.47, Rating: 4.9)
Customer CUST006 is new. Using content-based filtering.

Recommendations for new customer CUST006:
1. Breanna Turner (Score: 0.73, Rating: 5.0)
2. Jamie Lang (Score: 0.72, Rating: 3.6)
3. Fred Lara (Score: 0.63, Rating: 4.2)

Detailed dish preferences for existing customer:
Preferred categories: {'Veges': 3, 'FastFoods': 2, 'Rice': 2, 'Desserts': 1, 'Rotis': 1}
Preferred subcategories: {'Cake': 1, 'Fries': 1, 'Jeera Rice': 1, 'Paneer': 1, 'Fried Rice': 1, 'Kulcha': 1, 'Burger': 1, 'Aloo': 1, 'Palak': 1}


In [2]:
import pandas as pd
import numpy as np
import json
import ast
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, r2_score, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

# Load and prepare data functions from the provided code
def load_data():
    customers = pd.read_csv('customers.csv')
    orders = pd.read_csv('orders.csv')
    dishes = pd.read_csv('dishes.csv')
    chefs = pd.read_csv('chefs.csv')
    
    # Convert string representations of lists to actual lists
    customers['specialties preference'] = customers['specialties preference'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    chefs['specialties'] = chefs['specialties'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    # Parse dish information in orders
    orders['dishes'] = orders['dishes'].apply(
        lambda x: json.loads(x.replace("'", "\"")) if isinstance(x, str) else x
    )
    
    return customers, orders, dishes, chefs

def content_based_recommendations(customer_id, customers, chefs, top_n=3):
    """
    Recommend chefs based on matching customer preferences with chef specialties.
    Used for new customers without order history.
    """
    customer = customers[customers['customer_id'] == customer_id].iloc[0]
    
    # Get customer's diet preference and cuisine specialties
    diet_pref = customer['preference']
    cuisine_prefs = customer['specialties preference']
    
    # Create a feature vector for each chef
    chef_features = []
    for _, chef in chefs.iterrows():
        # Calculate cuisine match score (how many customer preferences match chef specialties)
        cuisine_match = sum(cuisine in chef['specialties'] for cuisine in cuisine_prefs) if len(cuisine_prefs) > 0 else 0
        cuisine_match_ratio = cuisine_match / len(cuisine_prefs) if len(cuisine_prefs) > 0 else 0
        
        # Consider chef rating and experience
        chef_score = (chef['averageRating'] / 5) * 0.5 + (chef['experience'] / 10) * 0.2 + cuisine_match_ratio * 0.3
        
        chef_features.append({
            'chef_id': chef['chef_id'],
            'name': chef['name'],
            'score': chef_score,
            'cuisine_match': cuisine_match_ratio,
            'rating': chef['averageRating']
        })
    
    # Sort chefs by score
    chef_features.sort(key=lambda x: x['score'], reverse=True)
    
    return chef_features[:top_n]

def create_user_item_matrix(orders, customers, chefs):
    """
    Create a matrix where rows are customers and columns are chefs.
    Each cell contains the number of times a customer ordered from a chef.
    """
    # Count orders for each customer-chef pair
    customer_chef_counts = orders.groupby(['customer_id', 'chef_id']).size().reset_index(name='order_count')
    
    # Create pivot table
    user_item_matrix = customer_chef_counts.pivot(
        index='customer_id', 
        columns='chef_id', 
        values='order_count'
    ).fillna(0)
    
    return user_item_matrix

def collaborative_filtering(customer_id, user_item_matrix, customers, chefs, top_n=3):
    """
    Recommend chefs based on customer's previous orders and similarity to other customers.
    Used for existing customers with order history.
    """
    # Check if customer exists in matrix
    if customer_id not in user_item_matrix.index:
        return []
    
    # Calculate cosine similarity between users
    user_similarity = cosine_similarity(user_item_matrix)
    user_similarity_df = pd.DataFrame(
        user_similarity,
        index=user_item_matrix.index,
        columns=user_item_matrix.index
    )
    
    # Get similar users to our target customer
    similar_users = user_similarity_df[customer_id].sort_values(ascending=False)[1:6]  # Top 5 similar users
    
    # Which chefs has the customer already ordered from?
    customer_chefs = user_item_matrix.loc[customer_id]
    customer_chefs = set(customer_chefs[customer_chefs > 0].index)
    
    # Collect recommendations from similar users
    chef_scores = {}
    for similar_user, similarity in similar_users.items():
        if similarity <= 0:  # Skip negatively correlated users
            continue
            
        # Get chefs this similar user has ordered from
        user_chefs = user_item_matrix.loc[similar_user]
        user_chefs = set(user_chefs[user_chefs > 0].index)
        
        # Recommend chefs this similar user has ordered from but our target customer hasn't
        for chef_id in user_chefs - customer_chefs:
            if chef_id not in chef_scores:
                chef_scores[chef_id] = 0
            chef_scores[chef_id] += similarity * user_item_matrix.loc[similar_user, chef_id]
    
    # Sort chefs by score and get top_n
    recommended_chefs = sorted(chef_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    # Get chef details
    chef_details = []
    for chef_id, score in recommended_chefs:
        chef = chefs[chefs['chef_id'] == chef_id].iloc[0]
        chef_details.append({
            'chef_id': chef_id,
            'name': chef['name'],
            'score': score,
            'rating': chef['averageRating'],
            'specialties': chef['specialties']
        })
    
    return chef_details

def hybrid_recommendations(customer_id, customers, orders, dishes, chefs, top_n=3):
    """
    Hybrid recommendation system that combines collaborative and content-based filtering.
    """
    # Create user-item matrix for collaborative filtering
    user_item_matrix = create_user_item_matrix(orders, customers, chefs)
    
    # Check if customer has previous orders
    customer_orders = orders[orders['customer_id'] == customer_id]
    has_order_history = len(customer_orders) > 0
    
    if has_order_history:
        # Use collaborative filtering first
        collab_recommendations = collaborative_filtering(customer_id, user_item_matrix, customers, chefs, top_n)
        content_recommendations = content_based_recommendations(customer_id, customers, chefs, top_n)
        
        # Merge recommendations with weights (70% collaborative, 30% content-based)
        combined_recs = {}
        
        # Add collaborative recommendations with weight
        for rec in collab_recommendations:
            combined_recs[rec['chef_id']] = {
                'chef_id': rec['chef_id'],
                'name': rec['name'],
                'score': rec['score'] * 0.7,
                'rating': rec['rating']
            }
        
        # Add or update with content recommendations
        for rec in content_recommendations:
            if rec['chef_id'] in combined_recs:
                combined_recs[rec['chef_id']]['score'] += rec['score'] * 0.3
            else:
                combined_recs[rec['chef_id']] = {
                    'chef_id': rec['chef_id'],
                    'name': rec['name'],
                    'score': rec['score'] * 0.3,
                    'rating': rec['rating']
                }
        
        # Convert to list and sort
        recommendations = list(combined_recs.values())
        recommendations.sort(key=lambda x: x['score'], reverse=True)
        recommendations = recommendations[:top_n]
    else:
        # For new users, use content-based only
        recommendations = content_based_recommendations(customer_id, customers, chefs, top_n)
    
    return recommendations

def analyze_dish_preferences(customer_id, orders, dishes):
    """
    Analyze which categories and subcategories of dishes the customer prefers.
    """
    customer_orders = orders[orders['customer_id'] == customer_id]
    
    if len(customer_orders) == 0:
        return None
    
    # Extract all dish IDs from customer orders
    dish_ids = []
    for _, order in customer_orders.iterrows():
        order_dishes = order['dishes']
        for dish_item in order_dishes:
            dish_ids.append(dish_item['dish'])
    
    # Get dish details
    customer_dishes = dishes[dishes['dish_id'].isin(dish_ids)]
    
    # Count categories and subcategories
    category_counts = customer_dishes['category'].value_counts().to_dict()
    subcategory_counts = customer_dishes['subCategory'].value_counts().to_dict()
    
    return {
        'categories': category_counts,
        'subcategories': subcategory_counts
    }

def enhanced_recommendations(customer_id, customers, orders, dishes, chefs, top_n=3):
    """
    Enhanced recommendation system that considers dish preferences.
    """
    base_recommendations = hybrid_recommendations(customer_id, customers, orders, dishes, chefs, top_n)
    dish_prefs = analyze_dish_preferences(customer_id, orders, dishes)
    
    if dish_prefs is None:
        return base_recommendations
    
    # Adjust chef scores based on their expertise in preferred dish categories
    for i, rec in enumerate(base_recommendations):
        chef_id = rec['chef_id']
        chef_dishes = dishes[dishes['chef_id'] == chef_id]
        
        # Calculate category match score
        category_match = 0
        total_categories = sum(dish_prefs['categories'].values())
        
        for category, count in dish_prefs['categories'].items():
            chef_category_count = len(chef_dishes[chef_dishes['category'] == category])
            if chef_category_count > 0:
                category_match += (count / total_categories) * (chef_category_count / len(chef_dishes))
        
        # Adjust recommendation score
        base_recommendations[i]['score'] = base_recommendations[i]['score'] * (1 + category_match)
    
    # Re-sort based on adjusted scores
    base_recommendations.sort(key=lambda x: x['score'], reverse=True)
    
    return base_recommendations[:top_n]

# New functions for data splitting and evaluation

def split_orders_train_test(orders, test_size=0.2, random_state=42):
    """
    Split orders into training and test sets while ensuring
    we have some history for all customers in the training set.
    """
    # Get unique customers
    unique_customers = orders['customer_id'].unique()
    
    # Dictionary to store train/test indices for each customer
    train_indices = []
    test_indices = []
    
    for customer_id in unique_customers:
        customer_orders = orders[orders['customer_id'] == customer_id]
        
        # If customer has only 1 order, keep it in training
        if len(customer_orders) == 1:
            train_indices.extend(customer_orders.index.tolist())
            continue
        
        # Otherwise split orders for this customer
        cust_train, cust_test = train_test_split(
            customer_orders.index, 
            test_size=test_size,
            random_state=random_state
        )
        
        train_indices.extend(cust_train)
        test_indices.extend(cust_test)
    
    # Create training and test DataFrames
    train_orders = orders.loc[train_indices].copy()
    test_orders = orders.loc[test_indices].copy()
    
    return train_orders, test_orders

def create_ground_truth(test_orders):
    """
    Create ground truth from test set.
    For each customer, collect the chefs they ordered from.
    """
    ground_truth = {}
    
    for _, order in test_orders.iterrows():
        customer_id = order['customer_id']
        chef_id = order['chef_id']
        
        if customer_id not in ground_truth:
            ground_truth[customer_id] = set()
        
        ground_truth[customer_id].add(chef_id)
    
    return ground_truth

def calculate_metrics(recommendations, ground_truth, top_n=3):
    """
    Calculate precision, recall, and F1 score for recommendations.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for customer_id, actual_chefs in ground_truth.items():
        if customer_id in recommendations:
            recommended_chefs = set([rec['chef_id'] for rec in recommendations[customer_id][:top_n]])
            tp = len(actual_chefs.intersection(recommended_chefs))
            fp = len(recommended_chefs - actual_chefs)
            fn = len(actual_chefs - recommended_chefs)
            
            true_positives += tp
            false_positives += fp
            false_negatives += fn
    
    # Calculate metrics
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

def calculate_rmse(recommendations, test_orders, dishes, chefs):
    """
    Calculate Root Mean Squared Error based on predicted chef ratings
    versus actual ratings customers would give (based on dish ratings).
    """
    predicted_ratings = []
    actual_ratings = []
    
    for _, order in test_orders.iterrows():
        customer_id = order['customer_id']
        chef_id = order['chef_id']
        
        # Skip if no recommendations for this customer
        if customer_id not in recommendations:
            continue
        
        # Find chef in recommendations if present
        chef_rec = None
        for rec in recommendations[customer_id]:
            if rec['chef_id'] == chef_id:
                chef_rec = rec
                break
        
        if chef_rec is None:
            continue
        
        # Get predicted rating (recommendation score normalized to 0-5 scale)
        pred_rating = min(5, max(1, chef_rec['score'] * (5/max(r['score'] for r in recommendations[customer_id]))))
        predicted_ratings.append(pred_rating)
        
        # Get actual rating from order dishes
        order_dishes = order['dishes']
        dish_ratings = []
        for dish_item in order_dishes:
            if 'rating' in dish_item and dish_item['rating'] is not None:
                dish_ratings.append(dish_item['rating'])
        
        # If we have dish ratings, use their average as actual rating, otherwise use chef's average rating
        if dish_ratings:
            actual_rating = sum(dish_ratings) / len(dish_ratings)
        else:
            chef_row = chefs[chefs['chef_id'] == chef_id].iloc[0]
            actual_rating = chef_row['averageRating']
        
        actual_ratings.append(actual_rating)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings)) if len(actual_ratings) > 0 else None
    
    # Calculate R2 score
    r2 = r2_score(actual_ratings, predicted_ratings) if len(actual_ratings) > 0 else None
    
    return {
        'rmse': rmse,
        'r2_score': r2,
        'n_samples': len(actual_ratings)
    }

def evaluate_recommendation_system():
    # Load data
    customers, orders, dishes, chefs = load_data()
    
    # Split into train and test sets
    train_orders, test_orders = split_orders_train_test(orders, test_size=0.3)
    
    # Create ground truth from test set
    ground_truth = create_ground_truth(test_orders)
    
    # Models to evaluate
    models = {
        'Content-Based': content_based_recommendations,
        'Collaborative Filtering': lambda cid, cust, chefs, top_n: collaborative_filtering(
            cid, create_user_item_matrix(train_orders, cust, chefs), cust, chefs, top_n
        ),
        'Hybrid': lambda cid, cust, chefs, top_n: hybrid_recommendations(
            cid, cust, train_orders, dishes, chefs, top_n
        ),
        'Enhanced': lambda cid, cust, chefs, top_n: enhanced_recommendations(
            cid, cust, train_orders, dishes, chefs, top_n
        )
    }
    
    # Results dictionary
    results = {}
    
    # Evaluate each model
    for model_name, model_func in models.items():
        print(f"Evaluating {model_name}...")
        
        start_time = time.time()
        recommendations = {}
        
        # Generate recommendations for users in test set
        test_customers = list(set(test_orders['customer_id'].tolist()))
        for customer_id in test_customers:
            try:
                recs = model_func(customer_id, customers, chefs, top_n=5)
                recommendations[customer_id] = recs
            except Exception as e:
                print(f"Error generating recommendations for customer {customer_id} with {model_name}: {e}")
        
        # Measure time
        execution_time = time.time() - start_time
        
        # Calculate metrics
        classification_metrics = calculate_metrics(recommendations, ground_truth, top_n=3)
        regression_metrics = calculate_rmse(recommendations, test_orders, dishes, chefs)
        
        # Store results
        results[model_name] = {
            'precision': classification_metrics['precision'],
            'recall': classification_metrics['recall'],
            'f1_score': classification_metrics['f1_score'],
            'rmse': regression_metrics['rmse'],
            'r2_score': regression_metrics['r2_score'],
            'execution_time': execution_time,
            'n_recommendations': sum(len(recs) for recs in recommendations.values()),
            'n_customers': len(recommendations),
            'n_samples_rmse': regression_metrics['n_samples']
        }
        
        print(f"  - Precision: {classification_metrics['precision']:.4f}")
        print(f"  - Recall: {classification_metrics['recall']:.4f}")
        print(f"  - F1 Score: {classification_metrics['f1_score']:.4f}")
        if regression_metrics['rmse'] is not None:
            print(f"  - RMSE: {regression_metrics['rmse']:.4f}")
            print(f"  - R2 Score: {regression_metrics['r2_score']:.4f}")
        print(f"  - Execution Time: {execution_time:.2f} seconds")
        print(f"  - Customers with recommendations: {len(recommendations)}")
    
    # Create results DataFrame for easy comparison
    results_df = pd.DataFrame(results).T
    
    # Add coverage metric (percentage of test users with recommendations)
    results_df['coverage'] = results_df['n_customers'] / len(set(test_orders['customer_id']))
    
    return results_df, recommendations

# Run the evaluation
if __name__ == "__main__":
    print("Evaluating Chef Recommendation Systems")
    print("=====================================")
    
    results_df, recommendations = evaluate_recommendation_system()
    
    print("\nSummary of Results:")
    print("=================")
    print(results_df[['precision', 'recall', 'f1_score', 'rmse', 'r2_score', 'execution_time', 'coverage']])

Evaluating Chef Recommendation Systems
Evaluating Content-Based...
  - Precision: 0.1667
  - Recall: 0.2500
  - F1 Score: 0.2000
  - RMSE: 0.8175
  - R2 Score: -1.7663
  - Execution Time: 0.01 seconds
  - Customers with recommendations: 4
Evaluating Collaborative Filtering...


c:\Users\tejpa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


  - Precision: 0.2000
  - Recall: 0.1250
  - F1 Score: 0.1538
  - RMSE: 0.8000
  - R2 Score: nan
  - Execution Time: 0.05 seconds
  - Customers with recommendations: 4
Evaluating Hybrid...
  - Precision: 0.2500
  - Recall: 0.3750
  - F1 Score: 0.3000
  - RMSE: 2.1688
  - R2 Score: -16.4208
  - Execution Time: 0.06 seconds
  - Customers with recommendations: 4
Evaluating Enhanced...
  - Precision: 0.1667
  - Recall: 0.2500
  - F1 Score: 0.2000
  - RMSE: 2.2892
  - R2 Score: -18.4089
  - Execution Time: 0.14 seconds
  - Customers with recommendations: 4

Summary of Results:
                         precision  recall  f1_score      rmse   r2_score  \
Content-Based             0.166667   0.250  0.200000  0.817524  -1.766330   
Collaborative Filtering   0.200000   0.125  0.153846  0.800000        NaN   
Hybrid                    0.250000   0.375  0.300000  2.168781 -16.420788   
Enhanced                  0.166667   0.250  0.200000  2.289190 -18.408860   

                         execution_